# class

Pythonの型やclassについて学ぼう。

## `type`

インスタンスとしてオブジェクトの型を返すクラス。関数ではない。

In [1]:
a = 1
b = 1.0
print(type(a))
print(type(b))

<class 'int'>
<class 'float'>


`type(a)`で`a`の型`int`がインスタンスとなって返ってきた。

`__class__`属性からも同じものが取得できる。

In [2]:
print(a.__class__)
print(b.__class__)

<class 'int'>
<class 'float'>


`type(1)(3.5)`とかやると、`int(3.5)`と同じ意味になるので、`3`が返ってくる。

In [3]:
type(1)(3.5)

3

関数の型を見てみよう。

In [4]:
def f():
    pass

print(type(f))
print(type(print))

<class 'function'>
<class 'builtin_function_or_method'>


- ユーザ定義関数: `function`
- 組み込み関数: `builtin_function_or_method`

のよう。

では型の型はどうなっているかというと、

In [5]:
class A:
    pass

print(type(A))
print(type(int))

<class 'type'>
<class 'type'>


`type`となる。では`type`の型は

In [6]:
type(type)

type

`type`だね。`type`が関数ではなくクラスであることがわかる。

## `class`

新たな型の定義。

In [7]:
class A:
    pass

a = A()
print(type(a))

<class '__main__.A'>


`モジュール名.クラス名`が得られる。

In [8]:
__name__ # モジュール名

'__main__'

`type`を使ってクラスを定義することもできる。

`type("class_name", (object,), {"method_name": method})`とすると、`method_name`を持つ`object`を継承した`class_name`というクラスが定義される。

In [9]:
MyClass = type("MyClass", (), {"__call__": lambda self: print("Hello World!")})
my_instance = MyClass()
my_instance()

Hello World!


`class Name(object):`というのは、内部で`type("Name", (object,), {})`が呼ばれているということ。

## メタクラス

インスタンスがクラスとなるようなクラス。`type`はメタクラスの一つ。

参考: [Pythonのクラスとメタクラスまとめ](https://astropengu.in/posts/32/)

例として、キャメルケースでメソッド名を定義してもスネークケースに直してからクラスを定義してくれるメタクラスを作ってみる。

In [10]:
import re
def camel_to_snake(name):
    return re.sub(r'(?<!^)(?=[A-Z])', '_', name).lower()

class MyMeta(type):
    def __new__(cls, name, bases, dict):
        new_dict = {camel_to_snake(k): v for k, v in dict.items()}
        return super().__new__(cls, name, bases, new_dict)

これでクラスを定義してみる。

In [11]:
MyClass = MyMeta("my_class", (), {"MyMethod": lambda self: print("Hello World!")})
my_instance = MyClass()

try:
    my_instance.my_method()
except Exception as e:
    print(e)

try:
    my_instance.MyMethod()
except Exception as e:
    print(e)

Hello World!
'my_class' object has no attribute 'MyMethod'


`MyMethod`とキャメルケースでメソッド名を指定したが、`my_method`とスネークケースに直されている。  
`MyMethod`というメソッドを呼び出すとエラーになる。

なお、デフォルトのメタクラス`type`を使うと

In [12]:
MyClass = type("my_class", (), {"MyMethod": lambda self: print("Hello World!")})
my_instance = MyClass()

try:
    my_instance.my_method()
except Exception as e:
    print(e)

try:
    my_instance.MyMethod()
except Exception as e:
    print(e)

'my_class' object has no attribute 'my_method'
Hello World!


と`MyMethod`のままになる。

`class Name(object):`の形でクラスを定義する際に任意のメタクラス`Meta`を指定する場合は、`class Name(object, metaclass=Meta):`とする。

In [13]:
class MyClass(metaclass=MyMeta):
    def MyMethod(self):
        print("Hello World!")

my_instance = MyClass()

try:
    my_instance.my_method()
except Exception as e:
    print(e)

try:
    my_instance.MyMethod()
except Exception as e:
    print(e)

Hello World!
'MyClass' object has no attribute 'MyMethod'


## 抽象基底クラス

*Abstract Base Class, ABC*

継承した際に特定のメソッドの実装を強制するクラス。そのメソッドは抽象メソッドと呼ぶ。

`abc`モジュールを使う。

In [14]:
from abc import ABCMeta, abstractmethod

`abc.ABCMeta`をメタクラスとし、抽象メソッドにはデコレータで`abc.abstractmethod`を指定する。

In [15]:
class MyABC(metaclass=ABCMeta):
    @abstractmethod
    def my_method(self):
        pass

これを継承するときは、`my_method`を実装しないとエラーになる。

In [16]:
class MyClass(MyABC):
    pass

try:
    MyClass()
except Exception as e:
    print(e)

Can't instantiate abstract class MyClass with abstract method my_method


実装すれば普通に動く。

In [17]:
class MyClass(MyABC):
    def my_method(self):
        print("Hello World!")

try:
    MyClass()
except Exception as e:
    print(e)

ちなみに、抽象基底クラスはインスタンス化できない。

In [18]:
try:
    my_abc = MyABC()
except Exception as e:
    print(e)

Can't instantiate abstract class MyABC with abstract method my_method


抽象基底クラスは実装に制限をかけるもので、特殊な機能があるわけではない。コードの保守性を高めたい場合などに使う。